In [2]:
#r "nuget: YamlDotNet, 11.2.1"

using System;
using System.IO;
using System.Linq;
using YamlDotNet.Serialization;
using YamlDotNet.Serialization.NamingConventions;

Installed Packages YamlDotNet, 11.2.1

In [ ]:
class ConvertArchive
{
    static void Main()
    {
        string targetDir = Path.Combine(AppDomain.CurrentDomain.BaseDirectory, "./archive");
        ProcessMarkdownFiles(targetDir);
    }

    static void ProcessMarkdownFiles(string dir)
    {
        foreach (var file in Directory.GetFiles(dir, "*.*", SearchOption.AllDirectories)
                                      .Where(f => f.EndsWith(".md") || f.EndsWith(".mdx")))
        {
            ConvertFrontmatter(file);
        }
    }

    static void ConvertFrontmatter(string filePath)
    {
        string fileContent = File.ReadAllText(filePath);
        var parts = fileContent.Split(new[] { "---" }, StringSplitOptions.RemoveEmptyEntries);

        if (parts.Length < 2)
        {
            Console.WriteLine($"No frontmatter found in: {filePath}");
            return;
        }

        string yamlFrontmatter = parts[0].Trim();
        string content = string.Join("---", parts.Skip(1)).Trim();

        var deserializer = new DeserializerBuilder()
            .WithNamingConvention(CamelCaseNamingConvention.Instance)
            .Build();

        var frontmatterData = deserializer.Deserialize<object>(yamlFrontmatter);

        var frontmatterObject = $"export const frontmatter = {System.Text.Json.JsonSerializer.Serialize(frontmatterData, new System.Text.Json.JsonSerializerOptions { WriteIndented = true })};\n\n{content}";
        File.WriteAllText(filePath, frontmatterObject);

        Console.WriteLine($"Processed: {filePath}");
    }
}

In [ ]:
ConvertArchive.Main();